# Setup

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import random
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from skimage.metrics import structural_similarity as ssim
from PIL import Image

In [ ]:
from facedataset import FaceDataset
from metrics import psnr
from layers import GaussianBlur

In [ ]:
from model import Model
from classic_model import Classic

In [ ]:
def reset_seeds():
    random.seed(714)
    np.random.seed(714)
    torch.manual_seed(714)

reset_seeds()
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
batch_size = 4
input_size = 32
upscaling_factor = 8
output_size = upscaling_factor*input_size
use_gpu = torch.cuda.is_available()

num_examples = 4 # number of examples printed

In [ ]:
def load_model(name, upscaling_factor=2):
    model = Model(upscaling_factor)
    model.load_state_dict(torch.load(f"weights/{name}.ckpt", map_location=torch.device('cpu')))
    if use_gpu:
        model = model.cuda()
    
    return model

Loads the model from `weights/`, name is filename and shot_name is used to save results.

In [ ]:
name = "model_crit_mse_reg_disc_32"
short_name = "mse_disc"
# for chaining upscaling_factor may differ from the factor that needs to be given here
model = load_model(name, upscaling_factor)

# for comparison 
# name = ""
# short_name = "bicubic"
# model = Classic(output_size)


model.eval()
gb = GaussianBlur(3, 1.0)

## FFHQ

Evaluation on the FFHQ dataset.

In [ ]:
data_path = "data/small256x256"
test_set = FaceDataset(data_path, 65000, 70000, input_size, upscaling_factor, p_flip=0)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle=False, num_workers=4)

In [ ]:
data_path = "data/comp_test"
small_test_set = FaceDataset(data_path, 0, 100, input_size, upscaling_factor, p_flip=0)
small_test_loader = DataLoader(test_set, batch_size = batch_size, shuffle=False, num_workers=4)
# images without copyright

In [ ]:
# Class for chaining the model

class MultiModel(nn.Module):
    def __init__(self, model, amount):
        """
        amount : int
            how often to run model
        """
        super(MultiModel, self).__init__()
        
        self.model = model
        self.amount = amount
        self.gb = GaussianBlur(3, 1.0, use_gpu)
    
    def forward(self, x):
        out = x
        for i in range(self.amount-1):
            out = self.model(out)
            out = self.gb(out)
        out = self.model(out)
        
        return out

In [ ]:
multi = MultiModel(model, 3)

In [ ]:
def eval_model(model):
    """
    Evaluates model PSNR and SSIM on the complete test set.
    """
    
    model.eval()

    with torch.no_grad():
        val_psnr = []
        val_ssim = []

        for img, target in tqdm(test_loader):
            if use_gpu:
                img = img.cuda()
                target = target.cuda()
            out = model(img)
            val_psnr.append(psnr(out, target))
            out = out.cpu().permute(0, 2, 3, 1).numpy()
            target = target.cpu().permute(0, 2, 3, 1).numpy()
            for i in range(out.shape[0]):
                val_ssim.append(ssim(out[i], target[i], data_range=1, multichannel=True))


        val_psnr = torch.cat(val_psnr)
        val_ssim = torch.tensor(val_ssim)

    print(f"Mean PSNR {torch.mean(val_psnr):.2f} ± {torch.std(val_psnr):.2f}")
    print(f"Mean SSIM {torch.mean(val_ssim):.3f} ± {torch.std(val_ssim):.2f}")

In [ ]:
eval_model(model)

Saves the results of all upscalings on the small test set.

In [ ]:
with torch.no_grad():
    for k, (img, target) in tqdm(enumerate(small_test_loader), total=len(small_test_loader)):
        out = torch.clamp(model(img), 0, 1)
        psnrs = psnr(out, target)
        for i in range(batch_size):
            out_np = out[i].permute(1,2,0).numpy()
            target_np = target[i].permute(1,2,0).numpy()
            ssim_img = ssim(out_np, target_np, data_range=1, multichannel=True)
            img_pil = transforms.functional.to_pil_image(img[i])
            img_pil.save(f"results/{upscaling_factor}x/low_{k*batch_size+i}.png")
            target_pil = transforms.functional.to_pil_image(target[i])
            target_pil.save(f"results/{upscaling_factor}x/high_{k*batch_size+i}.png")
            out_pil = transforms.functional.to_pil_image(out[i])
            out_pil.save(f"results/{upscaling_factor}x/{short_name}_{k*batch_size+i}.png")
            with open(f"results/{upscaling_factor}x/{short_name}_{k*batch_size+i}.tex", "w") as file:
                file.write(rf"PSNR: {psnrs[i]:.2f}\\" + f"\nSSIM: {ssim_img:.3f}")

In [ ]:
b1 = (70, 90, 110, 130)
b2 = (60, 160, 100, 200)
for fn in [short_name, "low", "high"]:
    img = Image.open(f"results/{upscaling_factor}x/{fn}_29.png").resize((upscaling_factor*input_size, upscaling_factor*input_size))
    img.crop(b1).save(f"results/{upscaling_factor}x/crops/{fn}1_29.png")
    img.crop(b2).save(f"results/{upscaling_factor}x/crops/{fn}2_29.png")

# Set 5 & 14
 
To evaluate performance on set 5 and 14

In [ ]:
import os

In [ ]:
os.chdir("data")

In [ ]:
os.chdir("Set5")
# os.chdir("Set14")

In [ ]:
os.listdir()

In [ ]:
with torch.no_grad():
    val_psnr = []
    val_ssim = []

    for fn in os.listdir():
        target = transforms.functional.to_tensor(Image.open(fn)).unsqueeze(0)
        
        # the following images have an uneven size, which can not be halved
        # so the last row/column is chopped off
        if fn == "comic.png" or fn=="zebra.png":
            target = target[:,:,:-1,:]
        if fn == "ppt3.png":
            target = target[:,:,:,:-1]
        img = gb(target)[:,:,::2,::2]

        if use_gpu:
            img = img.cuda()
            target = target.cuda()
        #out = model(img)
        out = transforms.functional.to_pil_image(img.squeeze())
        out = out.resize((out.size[0]*2, out.size[1]*2), 3)
        out = transforms.functional.to_tensor(out).unsqueeze(0)
        val_psnr.append(psnr(out, target))
        out = out.cpu().permute(0, 2, 3, 1).numpy()
        target = target.cpu().permute(0, 2, 3, 1).numpy()
        for i in range(out.shape[0]):
            val_ssim.append(ssim(out[i], target[i], data_range=1, multichannel=True))


    val_psnr = torch.cat(val_psnr)
    val_ssim = torch.tensor(val_ssim)

    print(f"Mean PSNR {torch.mean(val_psnr):.2f} ± {torch.std(val_psnr):.2f}")
    print(f"Mean SSIM {torch.mean(val_ssim):.3f} ± {torch.std(val_ssim):.2f}")

In [ ]:
os.chdir("../Set14")
# os.chdir("../Set5")

In [ ]:
del target